In [28]:
#Importo la librerias
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree

from sklearn.model_selection import train_test_split 
import sklearn

In [29]:
#importo el dataset y hago revision de los datos

df = pd.read_csv("titles_filtrado.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            5850 non-null   int64  
 1   id                    5850 non-null   object 
 2   title                 5849 non-null   object 
 3   type                  5850 non-null   object 
 4   description           5832 non-null   object 
 5   release_year          5850 non-null   int64  
 6   age_certification     3231 non-null   object 
 7   runtime               5850 non-null   int64  
 8   genres                5850 non-null   object 
 9   production_countries  5850 non-null   object 
 10  seasons               2106 non-null   float64
 11  imdb_id               5447 non-null   object 
 12  imdb_score            5850 non-null   float64
 13  imdb_votes            5352 non-null   float64
 14  tmdb_popularity       5759 non-null   float64
 15  tmdb_score           

In [30]:
# Drop de columnas innecesarias
df.drop("id", inplace=True, axis=1)

df.drop("imdb_id", inplace=True, axis=1)

# Limpio `genres` y `production_countries`, devuelta tomando el primer elemento de de la lista que cada valor retenga
def parse_list(row):
    parsed = str(row)[1:-1].split(",")
    for i in range(len(parsed)):
        parsed[i] = parsed[i].strip()[1:-1]
        
    return parsed[0] if parsed[0] != '' else 'N/A'

df['genres_parsed'] = df['genres'].map(parse_list)
df['prod_countries_parsed'] = df['production_countries'].map(parse_list)
df[['genres_parsed', 'prod_countries_parsed']]

,genres_parsed,prod_countries_parsed
0,documentation,US
1,drama,US
2,drama,US
3,fantasy,GB
4,war,GB
...,...,...
5845,romance,NG
5846,drama,N/A
5847,comedy,CO
5848,N/A,US


In [31]:
# Añado la columna 'is_movie'
df['is_movie'] = df.type.map(lambda x: 0 if x == 'SHOW' else 1)

In [32]:
# Obtengo todos los valores nulos
df.isnull().sum()

Unnamed: 0                  0
title                       1
type                        0
description                18
release_year                0
age_certification        2619
runtime                     0
genres                      0
production_countries        0
seasons                  3744
imdb_score                  0
imdb_votes                498
tmdb_popularity            91
tmdb_score                  0
genres_parsed               0
prod_countries_parsed       0
is_movie                    0
dtype: int64

In [33]:
# Drop de columnas innecesarias
df.drop("title", axis=1, inplace=True)
df.drop("description", axis=1, inplace=True)

# Relleno valores nulos
df['age_certification'].fillna("N/A", inplace=True)
df['seasons'].fillna(0.0, inplace=True)
df['tmdb_popularity'].fillna(df['tmdb_popularity'].mean(), inplace=True)
df['imdb_votes'].fillna(df['imdb_votes'].mean(), inplace=True)


# Confirmo que no queden valores nulos
df.isnull().sum()

Unnamed: 0               0
type                     0
release_year             0
age_certification        0
runtime                  0
genres                   0
production_countries     0
seasons                  0
imdb_score               0
imdb_votes               0
tmdb_popularity          0
tmdb_score               0
genres_parsed            0
prod_countries_parsed    0
is_movie                 0
dtype: int64

In [34]:
# Trasnfroma las variables categoricas en numericas
all_data = pd.get_dummies(df[['age_certification', 'genres_parsed', 'prod_countries_parsed', 'imdb_votes', 'seasons', 'is_movie']])




In [35]:
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn import preprocessing
from sklearn import utils


X = all_data.copy()
y = df.pop('imdb_score')




In [80]:
#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.4, random_state = 42)

In [81]:
from sklearn.tree import DecisionTreeClassifier #Importo el modelo

clf = tree.DecisionTreeClassifier(max_depth=None, random_state = None) #Creo el modelo




In [82]:
#Entreno el modelo

clf = clf.fit(X_train, y_train)

In [83]:
y_train_pred = clf.predict(X_train) #Prediccion en Train
y_test_pred = clf.predict(X_test) #Prediccion en Test
print(X_train)

        imdb_votes  seasons  is_movie  age_certification_G  \
3195  50988.000000      0.0         1                    0   
5253  23439.382474      0.0         1                    0   
4103   2350.000000      0.0         1                    0   
1551    863.000000      0.0         1                    0   
5189   8706.000000      3.0         0                    0   
...            ...      ...       ...                  ...   
3772   1345.000000      0.0         1                    0   
5191   2590.000000      0.0         1                    0   
5226    535.000000      0.0         1                    0   
5390  23439.382474      1.0         0                    0   
860   22894.000000      6.0         0                    0   

      age_certification_N/A  age_certification_NC-17  age_certification_PG  \
3195                      0                        0                     0   
5253                      1                        0                     0   
4103                 

In [84]:
from sklearn.metrics import accuracy_score

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:', test_accuracy)

% de aciertos sobre el set de entrenamiento: 0.9971509971509972
% de aciertos sobre el set de evaluación: 0.10811965811965812
